## Installing dependencies and loading packages

In [19]:

#pip install -q openai
#pip install -q llama-index
#pip install -q llama-index-experimental
#pip install -q pypdf
#pip install -q docx2txt

zsh:1: /Users/hannes/Documents/Programming/GitHub/GenIpA/GenIpa/bin/pip: bad interpreter: /Users/hannes/Documents/Programming/GitHub/GenIpA/.conda/bin/python: no such file or directory
zsh:1: /Users/hannes/Documents/Programming/GitHub/GenIpA/GenIpa/bin/pip: bad interpreter: /Users/hannes/Documents/Programming/GitHub/GenIpA/.conda/bin/python: no such file or directory
zsh:1: /Users/hannes/Documents/Programming/GitHub/GenIpA/GenIpa/bin/pip: bad interpreter: /Users/hannes/Documents/Programming/GitHub/GenIpA/.conda/bin/python: no such file or directory


In [1]:
import os
import openai

os.environ["OPENAI_API_KEY"] = "sk-proj-ati0Co2Vt8anHxAf9emxT3BlbkFJzBBqjuNyNos5W71ryVtc"
openai.api_key = os.environ["OPENAI_API_KEY"]

import sys
import shutil
import glob
import logging
from pathlib import Path

import warnings
warnings.filterwarnings('ignore')

import pandas as pd

## Llamaindex readers
from llama_index.core import SimpleDirectoryReader

## LlamaIndex Index Types
from llama_index.core import ListIndex
from llama_index.core import VectorStoreIndex
from llama_index.core import TreeIndex
from llama_index.core import KeywordTableIndex
from llama_index.core import SimpleKeywordTableIndex
from llama_index.core import DocumentSummaryIndex
from llama_index.core import KnowledgeGraphIndex
from llama_index.experimental.query_engine import PandasQueryEngine

## LlamaIndex Context Managers
from llama_index.core import StorageContext
from llama_index.core import load_index_from_storage
from llama_index.core.response_synthesizers import get_response_synthesizer
from llama_index.core.response_synthesizers import ResponseMode
from llama_index.core.schema import Node

## LlamaIndex Callbacks
from llama_index.core.callbacks import CallbackManager
from llama_index.core.callbacks import LlamaDebugHandler

In [2]:
from llama_index.core import Settings
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding

model="gpt-3.5-turbo"
#model="gpt-4"
#model="gpt-4-turbo"

Settings.embed_model = OpenAIEmbedding(model="text-embedding-ada-002")
Settings.llm = OpenAI(temperature=0, 
                      model=model, 
                      #max_tokens=512
                      PRESENCE_PENALTY=-2,
                      TOP_P=1,
                     )

## Defining directories

In [3]:
DOCS_DIR = "data/"
PERSIST_DIR = "index/"

print(f"Current dir: {os.getcwd()}")

if not os.path.exists(DOCS_DIR):
  os.mkdir(DOCS_DIR)
docs = os.listdir(DOCS_DIR)
docs = [d for d in docs]
docs.sort()
print(f"Files in {DOCS_DIR}")
for doc in docs:
    print(doc)

Current dir: /Users/hannes/Documents/Programming/GitHub/GenIpA
Files in data/
.DS_Store
en_UK.txt
sv.txt


## Creating indexes

In [4]:
from llama_index.core.node_parser import SentenceSplitter
splitter = SentenceSplitter(chunk_size=2048)

response_synthesizer = get_response_synthesizer(
    response_mode="tree_summarize", use_async=True
)

In [12]:
def create_retrieve_index(index_path, docs_path, index_type):
    if not os.path.exists(index_path):
        print(f"Creating Directory {index_path}")
        os.mkdir(index_path)
    if os.listdir(index_path) == []:
        print("Loading Documents...")
        documents = SimpleDirectoryReader(input_files=[f"{docs_path}{'sv.txt'}"]).load_data()
        print("Creating Index...")
        index = index_type.from_documents(documents,
                                          transformations=[splitter],
                                          #response_synthesizer=response_synthesizer,
                                          show_progress=True,
                                          )
        print("Persisting Index...")
        index.storage_context.persist(persist_dir=index_path)
        print("Done!")
    else:
        print("Reading from Index...")
        index = load_index_from_storage(storage_context=StorageContext.from_defaults(persist_dir=index_path))
        print("Done!")
    return index

In [13]:
os.listdir(DOCS_DIR)

['en_UK.txt', 'sv.txt', '.DS_Store']

### Creating vector store indexes for swedish dictionary

Why Vector store indexes? Efficient

In [14]:
VECTORINDEXDIR = PERSIST_DIR + 'VectorStoreIndex'
vectorstoreindex = create_retrieve_index(VECTORINDEXDIR, DOCS_DIR, VectorStoreIndex)

Loading Documents...
Creating Index...


Generating embeddings: 100%|██████████| 235/235 [00:06<00:00, 35.27it/s]


Persisting Index...
Done!


### Creating vector store indexes for UK dictionary

In [116]:
VECTORINDEXDIR = PERSIST_DIR + 'VectorStoreIndex'
vectorstoreindex = create_retrieve_index(VECTORINDEXDIR, DOCS_DIR, VectorStoreIndex, 'en_UK.txt')

Reading from Index...
Done!


## Querying the vector base

In [118]:
query_engine = vectorstoreindex.as_query_engine(retriever_mode="embedding",
                                                response_mode="compact",
                                                verbose=True)

In [132]:
response = query_engine.query("Please transcribe this text to IPA: 'Jag bor ovan dig i huset' using the indexes" ) 
print(response)

/jɑːg bʊr ²'uːvɑːn dɪg iː ²hʉːsɛt/


In [135]:
response = query_engine.query("Transkribera denna text till IPA: 'Jag är ovan vid att spela fotboll'") 
print(response)

/jaːg eːr ʊˈvɑn vɪd at ˈspeːla ˈfɔtbɔl/


In [115]:
response = query_engine.query("Please transcribe this text to IPA: 'Jag spelar fotboll', ignoring your own knowledge") 
print(response)

/jɑː spelar f'ɔtːbɔl/


In [129]:
response = query_engine.query("Please transcribe this text to IPA: 'yesterday I waxed my car', using the indexes") 
print(response)

/j'ɛstɛrdɛj aɪ w'ækst maɪ k'ɑːr/


In [83]:
def get_completion(prompt, model="gpt-3.5-turbo", temperature=0):
    messages = [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": prompt},
        ]
    response = openai.chat.completions.create(model=model,
                                              messages=messages,
                                              temperature=temperature
                                              
                                              )
    return response.choices[0].message.content

In [120]:
prompt = f"""
"Please transcribe this text to IPA using the indexes: yesterday I read a book on water"
"""
response = get_completion(prompt, temperature=0.5)
print(response)

Sure! Here is the transcription of the text "yesterday I read a book on water" using the International Phonetic Alphabet (IPA):

/jɛstərˈdeɪ aɪ rɛd ə bʊk ɒn ˈwɔtər/


In [85]:
prompt = f"""
"Please transcribe this text to IPA in american english: yesterday I read a book on water"
"""
response = get_completion(prompt, temperature=0.5)
print(response)

Sure! Here is the transcription of the text "yesterday I read a book on water" in American English IPA:

/jɛstɚdeɪ aɪ rɛd ə bʊk ɑn ˈwɔtər/


NameError: name 'ɐ' is not defined